In [1]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [2]:
pip install efficientnet

In [3]:
import datetime
import os
import numpy as np
import tensorflow as tf

from efficientnet.keras import EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
from keras import backend as K
from tensorflow.keras.layers import Activation, Add, BatchNormalization, concatenate, Conv2D, Conv2DTranspose
from tensorflow.keras.layers import Dropout, LeakyReLU, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [4]:
np.random.seed = 1

dataset = "DRIVE"

main_path = f"/content/gdrive/My Drive/Data/{dataset}"

In [5]:
training_images = f"{main_path}/patches/images/train/"
training_labels = f"{main_path}/patches/labels/train/"

train_img = next(os.walk(training_images))[2]
train_lbs = next(os.walk(training_labels))[2]

train_img.sort()
train_lbs.sort()

In [6]:
X_train = np.concatenate([np.load(training_images + file_id)["arr_0"] for file_id in train_img], axis=0)
Y_train = np.concatenate([np.load(training_labels + file_id)["arr_0"] for file_id in train_lbs], axis=0)

X_train = X_train / 255
Y_train = Y_train / 255

In [7]:
# Y_train = Y_train.astype("float64")

In [8]:
print(np.max(X_train))
print(np.min(X_train))
print(np.max(Y_train))
print(np.min(Y_train))

1.0
0.0
1.0
0.0


In [9]:
X_train.shape

(80000, 64, 64, 3)

In [10]:
_, h, w, c = X_train.shape

input_shape = (h, w, c)

In [11]:
def residual_block(x, filters):
    y = Conv2D(filters, (3, 3), strides=(1, 1), padding="same")(x)
    y = LeakyReLU(alpha=0)(y)
    y = BatchNormalization()(y)

    y = Conv2D(filters, (3, 3), strides=(1, 1), padding="same")(y)
    y = LeakyReLU(alpha=0)(y)
    y = BatchNormalization()(y)

    out = Add()([x, y])
    return out

In [12]:
def reccurent_block(x, filters, a=0):
    bn_axis = 3

    x1 = Conv2D(filters, (3, 3), strides=(1, 1), padding="same")(x)
    x1 = LeakyReLU(alpha=a)(x1)

    x2 = Conv2D(filters, (3, 3), strides=(1, 1), padding="same")(x1)              
    x2 = LeakyReLU(alpha=a)(x2)
    x12 = Add()([x1, x2])

    x3 = Conv2D(filters, (3, 3), strides=(1, 1), padding="same")(x12) 
    x3 = LeakyReLU(alpha=a)(x3)
    x13 = Add()([x1, x3])

    x4 = Conv2D(filters, (3, 3), strides=(1, 1), padding="same")(x13)
    x4 = LeakyReLU(alpha=a)(x4)
    x14 = Add()([x1, x4])

    x = LeakyReLU(alpha=a)(x14)
    x = BatchNormalization()(x)
    return x

In [13]:
def efficientB4ResUnet(input_shape):
    encoder = EfficientNetB4(include_top=False, weights="imagenet", input_shape=input_shape)

    # contracting level 4
    input = encoder.input
    en4 = Conv2D(24, (3, 3), strides=(1, 1), padding="same", activation=None)(input)
    en4 = LeakyReLU(alpha=0)(en4)
    en4 = BatchNormalization()(en4)

    # contracting level 3
    en3 = encoder.layers[25].output

    # contracting level 2
    en2 = encoder.layers[83].output

    # contracting level 1
    en1 = encoder.layers[141].output

    # middle level 0
    en0 = encoder.layers[317].output
    cv0 = residual_block(en0, 160)
    cv0 = residual_block(cv0, 160)
    up0 = Conv2DTranspose(56, (2, 2), strides=(2, 2), padding="same", name="upconvolution_1")(cv0)

    # expandsion level 1
    cv1 = concatenate([up0, en1])
    cv1 = residual_block(cv1, 112)
    up1 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same", name="upconvolution_2")(cv1)

    # expandsion level 2
    cv2 = concatenate([up1, en2])
    cv2 = residual_block(cv2, 64)
    up2 = Conv2DTranspose(24, (2, 2), strides=(2, 2), padding="same", name="upconvolution_3")(cv2)

    # expandsion level 3
    cv3 = concatenate([up2, en3])
    cv3 = residual_block(cv3, 48)
    up3 = Conv2DTranspose(24, (2, 2), strides=(2, 2), padding="same", name="upconvolution_4")(cv3)

    # expandsion level 4
    cv4 = concatenate([up3, en4])
    cv4 = residual_block(cv4, 48)

    out = Conv2D(1, (1, 1), activation="sigmoid")(cv4)

    model = tf.keras.Model(inputs=[input], outputs=[out])
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [14]:
def efficientB5ResUnet(input_shape):
    encoder = EfficientNetB5(include_top=False, weights="imagenet", input_shape=input_shape)

    # contracting level 4
    input = encoder.input
    en4 = Conv2D(24, (3, 3), strides=(1, 1), padding="same", activation=None)(input)
    en4 = LeakyReLU(alpha=0)(en4)
    en4 = BatchNormalization()(en4)

    # contracting level 3
    en3 = encoder.layers[37].output

    # contracting level 2
    en2 = encoder.layers[110].output

    # contracting level 1
    en1 = encoder.layers[183].output

    # middle level 0
    en0 = encoder.layers[389].output
    cv0 = residual_block(en0, 176)
    cv0 = residual_block(cv0, 176)
    up0 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same", name="upconvolution_1")(cv0)

    # expandsion level 1
    cv1 = concatenate([up0, en1])
    cv1 = residual_block(cv1, 128)
    up1 = Conv2DTranspose(40, (2, 2), strides=(2, 2), padding="same", name="upconvolution_2")(cv1)

    # expandsion level 2
    cv2 = concatenate([up1, en2])
    cv2 = residual_block(cv2, 80)
    up2 = Conv2DTranspose(24, (2, 2), strides=(2, 2), padding="same", name="upconvolution_3")(cv2)

    # expandsion level 3
    cv3 = concatenate([up2, en3])
    cv3 = residual_block(cv3, 48)
    up3 = Conv2DTranspose(24, (2, 2), strides=(2, 2), padding="same", name="upconvolution_4")(cv3)

    # expandsion level 4
    cv4 = concatenate([up3, en4])
    cv4 = residual_block(cv4, 48)

    out = Conv2D(1, (1, 1), activation="sigmoid")(cv4)

    model = tf.keras.Model(inputs=[input], outputs=[out])
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [15]:
def efficientB6ResUnet(input_shape):
    encoder = EfficientNetB6(include_top=False, weights="imagenet", input_shape=input_shape)

    # contracting level 4
    input = encoder.input
    en4 = Conv2D(32, (3, 3), strides=(1, 1), padding="same", activation=None)(input)
    en4 = LeakyReLU(alpha=0)(en4)
    en4 = BatchNormalization()(en4)

    # contracting level 3
    en3 = encoder.layers[37].output

    # contracting level 2
    en2 = encoder.layers[125].output

    # contracting level 1
    en1 = encoder.layers[213].output

    # middle level 0
    en0 = encoder.layers[449].output
    cv0 = residual_block(en0, 200)
    cv0 = residual_block(cv0, 200)
    up0 = Conv2DTranspose(72, (2, 2), strides=(2, 2), padding="same", name="upconvolution_1")(cv0)

    # expandsion level 1
    cv1 = concatenate([up0, en1])
    cv1 = residual_block(cv1, 144)
    up1 = Conv2DTranspose(40, (2, 2), strides=(2, 2), padding="same", name="upconvolution_2")(cv1)

    # expandsion level 2
    cv2 = concatenate([up1, en2])
    cv2 = residual_block(cv2, 80)
    up2 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same", name="upconvolution_3")(cv2)

    # expandsion level 3
    cv3 = concatenate([up2, en3])
    cv3 = residual_block(cv3, 64)
    up3 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same", name="upconvolution_4")(cv3)

    # expandsion level 4
    cv4 = concatenate([up3, en4])
    cv4 = residual_block(cv4, 64)

    out = Conv2D(1, (1, 1), activation="sigmoid")(cv4)

    model = tf.keras.Model(inputs=[input], outputs=[out])
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [16]:
def efficientB6R2Unet(input_shape):
    a = 0

    encoder = EfficientNetB6(include_top=False, weights="imagenet", input_shape=input_shape)

    # contracting level 4
    input = encoder.input
    en4 = Conv2D(32, (3, 3), strides=(1, 1), padding="same", activation=None)(input)
    en4 = LeakyReLU(alpha=a)(en4)
    en4 = BatchNormalization()(en4)

    # contracting level 3
    en3 = encoder.layers[37].output

    # contracting level 2
    en2 = encoder.layers[125].output

    # contracting level 1
    en1 = encoder.layers[213].output

    # middle level 0
    en0 = encoder.layers[449].output
    cv0 = reccurent_block(en0, 200, a)
    up0 = Conv2DTranspose(200, (2, 2), strides=(2, 2), padding="same", name="upconvolution_1")(cv0)

    # expandsion level 1
    cv1 = concatenate([up0, en1])
    cv1 = reccurent_block(cv1, 144, a)
    up1 = Conv2DTranspose(144, (2, 2), strides=(2, 2), padding="same", name="upconvolution_2")(cv1)

    # expandsion level 2
    cv2 = concatenate([up1, en2])
    cv2 = reccurent_block(cv2, 80, a)
    up2 = Conv2DTranspose(80, (2, 2), strides=(2, 2), padding="same", name="upconvolution_3")(cv2)

    # expandsion level 3
    cv3 = concatenate([up2, en3])
    cv3 = reccurent_block(cv3, 64, a)
    up3 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same", name="upconvolution_4")(cv3)

    # expandsion level 4
    cv4 = concatenate([up3, en4])
    cv4 = reccurent_block(cv4, 32, a)

    out = Conv2D(1, (1, 1), activation="sigmoid")(cv4)

    model = tf.keras.Model(inputs=[input], outputs=[out])
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [17]:
def efficientB7R2Unet(input_shape):
    a = 0

    encoder = EfficientNetB7(include_top=False, weights="imagenet", input_shape=input_shape)

    # contracting level 4
    input = encoder.input
    en4 = reccurent_block(input, 32, a)

    # contracting level 3
    en3 = encoder.layers[49].output

    # contracting level 2
    en2 = encoder.layers[152].output

    # contracting level 1
    en1 = encoder.layers[255].output

    # middle level 0
    en0 = encoder.layers[551].output
    cv0 = reccurent_block(en0, 224, a)
    up0 = Conv2DTranspose(224, (2, 2), strides=(2, 2), padding="same", name="upconvolution_1")(cv0)

    # expandsion level 1
    cv1 = concatenate([up0, en1])
    cv1 = reccurent_block(cv1, 160, a)
    up1 = Conv2DTranspose(160, (2, 2), strides=(2, 2), padding="same", name="upconvolution_2")(cv1)

    # expandsion level 2
    cv2 = concatenate([up1, en2])
    cv2 = reccurent_block(cv2, 96, a)
    up2 = Conv2DTranspose(96, (2, 2), strides=(2, 2), padding="same", name="upconvolution_3")(cv2)

    # expandsion level 3
    cv3 = concatenate([up2, en3])
    cv3 = reccurent_block(cv3, 64, a)
    up3 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same", name="upconvolution_4")(cv3)

    # expandsion level 4
    cv4 = concatenate([up3, en4])
    cv4 = reccurent_block(cv4, 32, a)

    out = Conv2D(1, (1, 1), activation="sigmoid")(cv4)

    model = tf.keras.Model(inputs=[input], outputs=[out])
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [18]:
from tensorflow.keras.layers import Flatten

In [19]:
# Metric function
def dice_coef(y_true, y_pred):
    smooth = 1
    y_true_f = Flatten(y_true)
    y_pred_f = Flatten(y_pred)
    intersection = np.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)


# Loss funtion
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [20]:
def Rec_conv2d_bn(x, n_filter, nb_row, nb_col, border_mode="same", strides=(1, 1)):
    """Utility function to apply conv + BN."""

    # if K.image_dim_ordering() == 'tf':
    bn_axis = 3

    x1 = Conv2D(n_filter, (nb_row, nb_col), strides=strides, padding=border_mode, kernel_regularizer=l2(2e-3))(x)
    x1 = Activation("relu")(x1)

    x2 = Conv2D(n_filter, (nb_row, nb_col), strides=strides, padding=border_mode, kernel_regularizer=l2(2e-3))(x1)
    x2 = Activation("relu")(x2)
    x12 = Add()([x1, x2])

    x3 = Conv2D(n_filter, (nb_row, nb_col), strides=strides, padding=border_mode, kernel_regularizer=l2(2e-3))(x12)
    x3 = Activation("relu")(x3)
    x13 = Add()([x1, x3])

    x4 = Conv2D(n_filter, (nb_row, nb_col), strides=strides, padding="same", kernel_regularizer=l2(2e-3))(x13)
    x4 = Activation("relu")(x4)

    x14 = Add()([x1, x4])

    x = Activation("relu")(x14)

    x = BatchNormalization(axis=bn_axis)(x13)
    return x

In [21]:
def build_R2UNetED(input_shape):

    channel_axis = 3
    inputs = tf.keras.Input(input_shape)

    # first block
    x = Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
    rcnn_bn1 = Rec_conv2d_bn(x, 32, 3, 3)
    conv1_f = Add()([x, rcnn_bn1])
    pool1 = MaxPooling2D((2, 2))(conv1_f)
    conv_pool1 = Conv2D(64, (1, 1), activation="relu", padding="same")(pool1)

    # second RRCNN layer...
    rcnn_bn2 = Rec_conv2d_bn(conv_pool1, 64, 3, 3)
    conv2_f = Add()([conv_pool1, rcnn_bn2])
    pool2 = MaxPooling2D((2, 2))(conv2_f)
    conv_pool2 = Conv2D(128, (1, 1), activation="relu", padding="same")(pool2)

    # third RRCNN layer...
    rcnn_bn3 = Rec_conv2d_bn(conv_pool2, 128, 3, 3)
    conv3_f = Add()([conv_pool2, rcnn_bn3])
    pool3 = MaxPooling2D((2, 2))(conv3_f)
    conv_pool3 = Conv2D(256, (1, 1), activation="relu", padding="same")(pool3)

    # fourth RRCNN layer...
    rcnn_bn4 = Rec_conv2d_bn(conv_pool3, 256, 3, 3)
    conv4_f = Add()([conv_pool3, rcnn_bn4])

    # Decoder...

    up7_1 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding="same")(conv4_f)
    up7 = concatenate([up7_1, conv3_f], axis=channel_axis)
    up7 = Rec_conv2d_bn(up7, 128, 3, 3)
    conv7 = Conv2D(128, (3, 3), activation="relu", padding="same")(up7)
    # conv7 = Conv2D(128, (3, 3), activation="relu", padding="same")(conv7)

    up8_1 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same")(conv7)
    up8 = concatenate([up8_1, conv2_f], axis=channel_axis)
    up8 = Rec_conv2d_bn(up8, 128, 3, 3)
    conv8 = Conv2D(64, (3, 3), activation="relu", padding="same")(up8)
    # conv8 = Conv2D(64, (3, 3), activation="relu", padding="same")(conv8)

    up9_1 = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding="same")(conv8)
    up9 = concatenate([up9_1, conv1_f], axis=channel_axis)
    up9 = Rec_conv2d_bn(up9, 128, 3, 3)
    conv9 = Conv2D(32, (3, 3), activation="relu", padding="same")(up9)
    # conv9 = Conv2D(32, (3, 3), activation="relu", padding="same")(conv9)

    conv10 = Conv2D(1, (1, 1), activation="sigmoid")(conv9)

    model = tf.keras.Model(inputs=[inputs], outputs=[conv10])

    # model.compile(optimizer=Adam(2e-4), loss=dice_coef_loss, metrics=[dice_coef, "acc", "mse"])

    model.compile(optimizer=Adam(3e-4), loss='binary_crossentropy',metrics = ['acc', 'mse'])

    return model

In [22]:
depth = 7
blocks = "R2"

In [23]:
# if depth == 1:
#     model = efficientB1R2Unet(input_shape)
# elif depth == 4:
#     model = efficientB4R2Unet(input_shape)
# elif depth == 5:
    # model = efficientB5R2Unet(input_shape)
# elif depth == 6:
#     model = efficientB6R2Unet(input_shape)
# elif depth == 7:
#     model = efficientB7R2Unet(input_shape)

In [24]:
# model = build_R2UNetED(input_shape)

In [25]:
model = tf.keras.models.load_model(f"/content/gdrive/My Drive/NN Models/{dataset}/model_DRIVE_EffB7_R2_2021-02-09 04:15:41")

In [26]:
# K.set_value(model.optimizer.learning_rate, 0.0001)
# K.set_value(model.optimizer.learning_rate, 0.00003)
K.set_value(model.optimizer.learning_rate, 0.00001)
# K.set_value(model.optimizer.learning_rate, 0.000003)

In [27]:
# model.summary()

In [28]:
model.fit(x=X_train, y=Y_train, batch_size=128, epochs=10, verbose=2)

Epoch 1/10
625/625 - 237s - loss: 0.0016 - accuracy: 0.9994
Epoch 2/10
625/625 - 216s - loss: 0.0016 - accuracy: 0.9994
Epoch 3/10
625/625 - 216s - loss: 0.0016 - accuracy: 0.9994
Epoch 4/10
625/625 - 215s - loss: 0.0016 - accuracy: 0.9994
Epoch 5/10
625/625 - 215s - loss: 0.0016 - accuracy: 0.9994
Epoch 6/10
625/625 - 215s - loss: 0.0016 - accuracy: 0.9994
Epoch 7/10
625/625 - 215s - loss: 0.0016 - accuracy: 0.9994
Epoch 8/10
625/625 - 215s - loss: 0.0016 - accuracy: 0.9994
Epoch 9/10
625/625 - 215s - loss: 0.0016 - accuracy: 0.9994
Epoch 10/10
625/625 - 215s - loss: 0.0016 - accuracy: 0.9994


In [29]:
# model.save(f"/content/gdrive/My Drive/NN Models/{dataset}/model_{dataset}_R2Unet_{str(datetime.datetime.now())[:-7]}")

In [30]:
model.save(f"/content/gdrive/My Drive/NN Models/{dataset}/model_{dataset}_EffB{depth}_{blocks}_{str(datetime.datetime.now())[:-7]}")

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/NN Models/DRIVE/model_DRIVE_EffB7_R2_2021-02-09 05:08:34/assets


In [31]:
print("\n-----------------------------------------------End of process-----------------------------------------------")


-----------------------------------------------End of process-----------------------------------------------
